# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
## import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2018.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2018/01/01,32055158,2018-01-01,1,2018
1,2018/01/04,29297265,2018-01-04,1,2018
2,2018/01/23,29357303,2018-01-23,1,2018
3,2018/01/04,29295270,2018-01-04,1,2018
4,2018/01/18,29339485,2018-01-18,1,2018


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [4]:
#to continue where previous notebook left off

x = df_ls.index("df_2018_12_G342")
x = x+1

In [8]:
output_df = {}
for grp in tqdm(df_ls[x:]):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/18 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2018_12_G343



0it [00:00, ?it/s]
1it [00:00,  2.89it/s]
2it [00:00,  3.28it/s]
3it [00:00,  3.01it/s]
4it [00:01,  2.90it/s]
5it [00:01,  2.78it/s]
6it [00:02,  2.79it/s]
7it [00:02,  2.72it/s]
8it [00:02,  2.74it/s]
9it [00:03,  2.73it/s]
10it [00:03,  2.69it/s]
11it [00:03,  2.68it/s]
12it [00:04,  1.86it/s]
13it [00:05,  2.21it/s]
14it [00:05,  2.29it/s]
15it [00:05,  2.47it/s]
16it [00:06,  2.50it/s]
17it [00:06,  2.52it/s]
18it [00:07,  2.56it/s]
19it [00:07,  2.69it/s]
20it [00:07,  2.83it/s]
21it [00:08,  2.60it/s]
22it [00:08,  2.65it/s]
23it [00:08,  2.69it/s]
24it [00:09,  2.71it/s]
25it [00:09,  2.62it/s]
26it [00:09,  2.63it/s]
27it [00:10,  2.07it/s]
28it [00:10,  2.42it/s]
29it [00:11,  2.50it/s]
30it [00:11,  2.30it/s]
31it [00:12,  2.65it/s]
32it [00:12,  2.67it/s]
33it [00:12,  2.52it/s]
34it [00:13,  2.73it/s]
35it [00:13,  2.72it/s]
36it [00:13,  2.72it/s]
37it [00:14,  2.73it/s]
38it [00:14,  2.68it/s]
39it [00:15,  2.82it/s]

Error here in ID30524355



40it [00:15,  2.60it/s]
41it [00:15,  2.72it/s]
42it [00:16,  2.68it/s]
43it [00:16,  2.84it/s]
44it [00:16,  2.67it/s]
45it [00:17,  2.69it/s]
46it [00:17,  2.69it/s]
47it [00:18,  1.67it/s]
48it [00:19,  1.97it/s]
49it [00:19,  1.99it/s]
50it [00:19,  2.41it/s]
51it [00:20,  2.43it/s]
52it [00:20,  2.50it/s]
53it [00:20,  2.54it/s]

...

Error here in ID30589056



91it [00:34,  2.69it/s]
92it [00:34,  2.75it/s]
93it [00:35,  2.74it/s]
94it [00:35,  2.56it/s]
95it [00:36,  2.71it/s]
96it [00:36,  2.72it/s]
97it [00:36,  2.61it/s]

Error here in ID30530864



98it [00:37,  2.59it/s]
99it [00:37,  2.75it/s]
100it [00:37,  2.64it/s]
100%|██████████| 18/18 [11:38<00:00, 38.79s/it]

dict_keys(['ABSTRACTdf_2018_12_G343', 'ABSTRACTdf_2018_12_G344', 'ABSTRACTdf_2018_12_G345', 'ABSTRACTdf_2018_12_G346', 'ABSTRACTdf_2018_12_G347', 'ABSTRACTdf_2018_12_G348', 'ABSTRACTdf_2018_12_G349', 'ABSTRACTdf_2018_12_G350', 'ABSTRACTdf_2018_12_G351', 'ABSTRACTdf_2018_12_G352', 'ABSTRACTdf_2018_12_G353', 'ABSTRACTdf_2018_12_G354', 'ABSTRACTdf_2018_12_G355', 'ABSTRACTdf_2018_12_G356', 'ABSTRACTdf_2018_12_G357', 'ABSTRACTdf_2018_12_G358', 'ABSTRACTdf_2018_12_G359', 'ABSTRACTdf_2018_12_G360'])
